In [1]:
# Importing
import pdb
import sys
from regressor import GASpyRegressor
import gpickle
import gplot
sys.path.insert(0, '../')
from gaspy.utils import vasp_settings_to_str

VASP_SETTINGS = vasp_settings_to_str({'gga': 'RP',
                                      'pp_version': '5.4',
                                      'encut': 350})

In [2]:
import copy
from tpot import TPOTRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [3]:
model_name = 'GP_around_TPOT'
features = ['coordcount']
outer_features = ['neighbors_coordcounts']
responses = ['energy']
blocks = ['adsorbate']
fingerprints = {'neighborcoord': '$processed_data.fp_final.neighborcoord'}

In [7]:
tpot = TPOTRegressor(
                     generations=4,
                     population_size=16,
                     verbosity=2,
                     random_state=42,
                    )
gp = GaussianProcessRegressor(
                              #kernel= 1.0*RBF(length_scale=0.05) \
                              #       +1.0*RBF(length_scale=0.2) \
                              #       +1.0*WhiteKernel(noise_level=0.05**2.0),
                              #n_restarts_optimizer=2,
                             )
H = GASpyRegressor(features=features, responses=responses,
                   blocks=blocks, vasp_settings=VASP_SETTINGS,
                   fingerprints=fingerprints, train_size=0.99)
H.fit_tpot(tpot, model_name=model_name)
H.fit_hierarchical(gp, 'fit_sk', outer_features, model_name=model_name)

Version 0.8.3 of tpot is outdated. Version 0.9.0 was released Wednesday September 27, 2017.


Optimization Progress:  38%|███▊      | 30/80 [00:27<00:52,  1.05s/pipeline]

Generation 1 - Current best internal CV score: 0.498408963114


Optimization Progress:  56%|█████▋    | 45/80 [00:46<00:44,  1.28s/pipeline]

Generation 2 - Current best internal CV score: 0.498408963114


...

Optimization Progress:  56%|█████▋    | 45/80 [00:25<00:16,  2.15pipeline/s]

Generation 2 - Current best internal CV score: 0.190960485295


Optimization Progress:  76%|███████▋  | 61/80 [00:39<00:16,  1.16pipeline/s]

Generation 3 - Current best internal CV score: 0.190329150094


Generation 4 - Current best internal CV score: 0.190329150094

Best pipeline: RidgeCV(LinearSVR(input_matrix, LinearSVR__C=15.0, LinearSVR__dual=DEFAULT, LinearSVR__epsilon=0.1, LinearSVR__loss=epsilon_insensitive, LinearSVR__tol=0.01))


In [8]:
gpickle.dump(H)